In [1]:
# Scientific computing
import pandas as pd
import numpy as np
import scipy

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Inspect data

In [2]:
df_full = pd.read_csv('data/heart_2020_cleaned.csv')
print(df_full.info())
df_full.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319795 entries, 0 to 319794
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   HeartDisease      319795 non-null  object 
 1   BMI               319795 non-null  float64
 2   Smoking           319795 non-null  object 
 3   AlcoholDrinking   319795 non-null  object 
 4   Stroke            319795 non-null  object 
 5   PhysicalHealth    319795 non-null  float64
 6   MentalHealth      319795 non-null  float64
 7   DiffWalking       319795 non-null  object 
 8   Sex               319795 non-null  object 
 9   AgeCategory       319795 non-null  object 
 10  Race              319795 non-null  object 
 11  Diabetic          319795 non-null  object 
 12  PhysicalActivity  319795 non-null  object 
 13  GenHealth         319795 non-null  object 
 14  SleepTime         319795 non-null  float64
 15  Asthma            319795 non-null  object 
 16  KidneyDisease     31

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


The dataset was cleaned, thanks to KAMIL PYTLAK.

In [5]:
# Number of features and instances
df_full.shape, df_full.columns.to_list()

((319795, 18),
 ['HeartDisease',
  'BMI',
  'Smoking',
  'AlcoholDrinking',
  'Stroke',
  'PhysicalHealth',
  'MentalHealth',
  'DiffWalking',
  'Sex',
  'AgeCategory',
  'Race',
  'Diabetic',
  'PhysicalActivity',
  'GenHealth',
  'SleepTime',
  'Asthma',
  'KidneyDisease',
  'SkinCancer'])

# Plans

### Mục tiêu:
- Phân tích được các yếu tố có thể liên quan đến tình trạng tim mạch của một người.
- Trực quan hóa các thông tin quan trọng
- Biến đổi data thành dạng dữ liệu thân thiện với học máy

### Thuộc tính đã có: có 17 thuộc tính độc lập:
- 'BMI': Continuous,
- 'Smoking': Categorical,
- 'AlcoholDrinking' : Categorical,
- 'Stroke': Categorical,
- 'PhysicalHealth': Continuous,
- 'MentalHealth': Continuous,
- 'DiffWalking': Categorical,
- 'Sex': Categorical,
- 'AgeCategory': Categorical,
- 'Race': Categorical,
- 'Diabetic': Categorical,
- 'PhysicalActivity': Categorical,
- 'GenHealth': Continuous,
- 'SleepTime': Continuous,
- 'Asthma': Categorical,
- 'KidneyDisease': Categorical,
- 'SkinCancer': Categorical

### Biến phụ thuộc:

Khả năng mắc bệnh tim của một người. Bộ dữ liệu của chúng ta không có sẵn số liệu này, thay vào đó là tình trạng mắc bệnh tim 'HeartDisease'.

### Mô tả kế hoạch:

Đầu tiên tôi sẽ tìm hiểu một vài thông tin thống kê của dataset như tỷ lệ mắc bệnh tim chung, phân phối của các thuộc tính, phát hiện các vấn đề có thể không phù hợp như outlier

Tiếp theo, tôi sẽ tìm hiểu xem những yếu tố nào ảnh hưởng đến khả năng mắc bệnh tim của một người, từ đó có thể đưa ra các yếu tố giúp dự đoán khả năng này. Tôi sẽ dùng một số công cụ kiểm định thống kê, trực quan hóa cho mục đích trên.

Một vài thuộc tính có thể sẽ khó có sẵn khi vào thực tế, có thể chúng ta sẽ cần chọn lựa ra các thuộc tính thực sự có thể ứng dụng.

Cuối cùng, tôi sẽ biến đổi data cho phù hợp với việc sử dụng các thuật toán học máy. Như chúng ta thấy, rất nhiều cột dữ liệu được thể hiện dưới loại dữ liệu object, chúng ta cần biến đổi chúng thành kiểu dữ liệu mà các thuật toán học máy có thể hiểu. Đồng thời, đối với một số cột dữ liệu như BMI, sleep time, tôi sẽ sử dụng binning để vừa thuận tiện cho việc tìm hiểu mối tương quan, và cũng có thể cải thiện kết quả mô hình

Chúng ta sẽ đi qua các bước
1. Statistical Analysis
1. Feature Selection
1. Data Transformation